In [0]:
!git clone https://github.com/zllrunning/face-parsing.PyTorch

Dont need these anymore


Need to Install few things

In [0]:
!pip install tensorboardX wandb
!unzip faceparsing.zip
!gdown https://drive.google.com/uc?id=154JgKpzCPW82qINcVieuPH3fZ2e0P812 -O faceparsing/79999_iter.pth

#for dataset
!git clone https://github.com/nalbert9/Facial-Keypoint-Detection.git

#load weight and biases
!wandb login c3c8bc8ee408b8d8d1e170586aa0c70bc4542720

# Init wandb
import wandb
wandb.init(project="deepannotation")


Import Libraries

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.model_zoo as modelzoo
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim



import os
import os.path as osp
import numpy as np
import pandas as pd
import glob
from PIL import Image
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorboardX import SummaryWriter

#Set cuda Driver 
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

#Load Writen Libraries

##To get Free annotation
from faceparsing.model import BiSeNet

#My Models
from faceparsing.Models import Net
from faceparsing.data_load import FacialKeypointsDataset,Normalize,Rescale,RandomCrop,ToTensor






Get Free annotation network Ready for face parsing

In [0]:
n_classes = 19
net = BiSeNet(n_classes=n_classes)
net.cuda()
save_pth = 'faceparsing/79999_iter.pth'
net.load_state_dict(torch.load(save_pth))
net.eval()

#For preparing Image as Input
to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

#Parsing face
with torch.no_grad():

  img = Image.open('im.jpeg')
  image = img.resize((512, 512), Image.BILINEAR)
  img = to_tensor(image)
  img = torch.unsqueeze(img, 0)
  img = img.cuda()
  out = net(img)[0]
  parsing = out.squeeze(0).cpu().numpy().argmax(0)
  print(parsing)
  # print(np.unique(parsing))

def FaceParser(img):
  image = img.resize((512, 512), Image.BILINEAR)
  img = to_tensor(image)
  img = torch.unsqueeze(img, 0)
  img = img.cuda()
  out = net(img)[0]
  parsing = out.squeeze(0).cpu().numpy().argmax(0)
  return parsing

def GetHaieAnnotationXY(parsing):
  #Use Canny edge detector from openCv to find hair edges
  hairx,hairy = np.where(parsing==17)
  edge = np.zeros((512,512))
  edge[hairx,hairy] = 255
  edges = cv2.Canny(np.uint8(edge),512,512)
  #Convert hair edgesarray to X,Y from many edges point
  edgex,edgey = np.where(edges==255)
  #Select 68 point randomly as hair annotation
  dots = np.zeros((2,68))
  for i in range(68):
    r = np.random.randint(0,len(edgex))
    dots[0,i] = edgex[r]
    dots[1,i] = edgey[r]
  return dots


  # plt.imshow(mask)
  #plt.imshow(edges)


  

First Simple Model Without hyper parameter tuning

In [0]:
#Init CNN Model
net = Net()
net = net.cuda()
wandb.watch(net, log="all")

Intiate Dataset and make Transformations

In [0]:
# define the data tranform
data_transform = transforms.Compose([Rescale(250),
                                     RandomCrop(224),
                                     Normalize(),
                                     ToTensor()])
# create the transformed dataset
transformed_dataset = FacialKeypointsDataset(csv_file='data/training_frames_keypoints.csv',
                                             root_dir='data/training/',
                                             transform=data_transform)
# load training data in batches
batch_size = 10
train_loader = DataLoader(transformed_dataset, 
                          batch_size=batch_size,
                          shuffle=True)
# create the test dataset
test_dataset = FacialKeypointsDataset(csv_file='data/test_frames_keypoints.csv',
                                             root_dir='data/test/',
                                             transform=data_transform)
# load test data in batches
test_loader = DataLoader(test_dataset, 
                          batch_size=batch_size,
                          shuffle=True)

Defining Loss Functions and optimizers. This can be separated later for cleaning perpose to lossClass.py

In [0]:
criterion = nn.MSELoss()
criterion = criterion.cuda(device)
optimizer = optim.Adam(net.parameters(), lr = 0.001)


Defining Train/Test Function

In [0]:
def train_net(n_epochs,net):

    # prepare the net for training
    
    # with torch.no_grad():

      
      net.train()
      training_loss = []
      for epoch in range(n_epochs):  # loop over the dataset multiple times
        
          running_loss = 0.0

          # train on batches of data, assumes you already have train_loader
          for batch_i, data in enumerate(train_loader):
            # get the input images and their corresponding labels
            images = data['image']
            key_pts = data['keypoints']
            images2 = images


            # flatten pts
            key_pts = key_pts.view(key_pts.size(0), -1)

            # convert variables to floats for regression loss
            key_pts = key_pts.type(torch.FloatTensor)
            key_pts = key_pts.to(device)
            images = images.type(torch.FloatTensor)
            images = images.to(device)

            # forward pass to get outputs
            output_pts = net(images)

            # calculate the loss between predicted and target keypoints
            loss = criterion(output_pts, key_pts)

            # zero the parameter (weight) gradients
            optimizer.zero_grad()
            
            # backward pass to calculate the weight gradients
            loss.backward()

            # update the weights
            optimizer.step()

            #Save All logs to Weights and Biases
            wandb.log({"Images" : images2})

            # print loss statistics
            running_loss += loss.item()
            if batch_i % 10 == 9:    # print every 10 batches
                print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1, batch_i+1, running_loss/10))
                running_loss = 0.0
          training_loss.append(running_loss)

      print('Finished Training')
      return training_loss


def net_sample_output():
    
    # iterate through the test dataset
    for i, sample in enumerate(test_loader):
        
        # get sample data: images and ground truth keypoints
        images = sample['image']
        key_pts = sample['keypoints']
        key_pts = key_pts.cuda(device)

        # convert images to FloatTensors
        images = images.type(torch.FloatTensor)
        images = images.cuda(device)

        # forward pass to get net output
        output_pts = net(images)
        
        # reshape to batch_size x 68 x 2 pts
        output_pts = output_pts.view(output_pts.size()[0], 68, -1)
        
        # break after first image is tested
        if i == 0:
            return images, output_pts, key_pts

        # wandb.log({"examples" : images})



Train Data

In [0]:
training_loss = train_net(n_epochs,net)

Data Visualization

In [0]:
def show_all_keypoints(image, predicted_key_pts, gt_pts=None):
    """Show image with predicted keypoints"""
    fig=plt.figure(figsize=(9, 8), dpi= 80, facecolor='w', edgecolor='k')    
    # image is grayscale
    plt.imshow(image, cmap='gray')
    plt.scatter(predicted_key_pts[:, 0], predicted_key_pts[:, 1], s=20, marker='.', c='m')
    # plot ground truth points as green pts
    if gt_pts is not None:
        plt.scatter(gt_pts[:, 0], gt_pts[:, 1], s=20, marker='.', c='g')



# visualize the output
# by default this shows a batch of 10 images
def visualize_output(test_images, test_outputs, gt_pts=None, batch_size=10):

    for i in range(batch_size):
        plt.figure(figsize=(20,10))
        ax = plt.subplot(1, batch_size, i+1)

        # un-transform the image data
        image = test_images[i].data   # get the image from it's Variable wrapper
        image = image.numpy()   # convert to numpy array from a Tensor
        image = np.transpose(image, (1, 2, 0))   # transpose to go from torch to numpy image

        # un-transform the predicted key_pts data
        predicted_key_pts = test_outputs[i].data
        predicted_key_pts = predicted_key_pts.numpy()
        # undo normalization of keypoints  
        predicted_key_pts = predicted_key_pts*50.0+100
        
        # plot ground truth points for comparison, if they exist
        ground_truth_pts = None
        if gt_pts is not None:
            ground_truth_pts = gt_pts[i]         
            ground_truth_pts = ground_truth_pts*50.0+100
        
        # call show_all_keypoints
        show_all_keypoints(np.squeeze(image), predicted_key_pts, ground_truth_pts)
            
        plt.axis('off')

    plt.show()



Test Data

In [0]:
# returns: test images, test predicted keypoints, test ground truth keypoints
test_images, test_outputs, gt_pts = net_sample_output()
# print out the dimensions of the data to see if they make sense
print(test_images.data.size())
print(test_outputs.data.size())
print(gt_pts.size())
    
visualize_output(test_images.cpu(), test_outputs.cpu(), gt_pts.cpu())


#################################################
# Start Real code
Building resNet


Load data
